In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer 
# from sklearn.decomposition import TruncatedSVD
# from sklearn.decomposition import NMF
# from sklearn.metrics.pairwise import cosine_similarity

# from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# Load data
# use skip initialspace in case headers has hidden space

df = pd.read_csv("C:/Users/storm/Desktop/METIS/Project/Project 5_RS/Dataset/sephora_SG_FIREFOX_dataset.csv")



In [3]:
df.head()

,Label,URL,brand,name,price,rank,skin_type,ingredients
0,cleanser-and-exfoliator,https://www.sephora.sg/products/fresh-soy-face...,FRESH,Soy Face Cleanser,$113.00,4.5,"Skin Type: Combination, Dry, Normal, Oily, Sen...","Aqua (Water), Coco-Glucoside, Glycerin, Butyle..."
1,cleanser-and-exfoliator,https://www.sephora.sg/products/indie-lee-brig...,INDIE LEE,Brightening Cleanser,$50.00,4.5,"Skin Type: Combination, Dry, Normal, Oily, Sen...","Water (Aqua/Eau), Decyl Glucoside (Veg. Oils &..."
2,cleanser-and-exfoliator,https://www.sephora.sg/products/tarte-sea-deep...,TARTE,Sea Deep Dive Makeup Removing Gel Cleanser,$40.00,4.5,"Skin Type: Combination, Dry, Normal, Oily, Sen...","Water/Aqua/Eau, cocamidopropyl betaine, disodi..."
3,cleanser-and-exfoliator,https://www.sephora.sg/products/fresh-rose-cle...,FRESH,Rose Cleansing Foam • 120ml,$47.00,4.0,"Skin Type: Combination, Dry, Normal, Oily, Sen...","Aqua (Water), Butylene Glycol, Decyl Glucoside..."
4,cleanser-and-exfoliator,https://www.sephora.sg/products/first-aid-beau...,FIRST AID BEAUTY,Face Cleanser,$55.00,4.5,Function: Face,Allantoin and Aloe Barbadensis Leaf Extract: H...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542 entries, 0 to 541
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Label        542 non-null    object 
 1   URL          542 non-null    object 
 2   brand        542 non-null    object 
 3   name         542 non-null    object 
 4   price        542 non-null    object 
 5   rank         542 non-null    float64
 6   skin_type    536 non-null    object 
 7   ingredients  542 non-null    object 
dtypes: float64(1), object(7)
memory usage: 34.0+ KB


In [5]:
#drop rows with NAN 

df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 536 entries, 0 to 541
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Label        536 non-null    object 
 1   URL          536 non-null    object 
 2   brand        536 non-null    object 
 3   name         536 non-null    object 
 4   price        536 non-null    object 
 5   rank         536 non-null    float64
 6   skin_type    536 non-null    object 
 7   ingredients  536 non-null    object 
dtypes: float64(1), object(7)
memory usage: 37.7+ KB


In [6]:
df.reset_index(drop=True, inplace = True)

In [7]:
'''
certain cells under skin_type column contain wrong scraped info e.g

Function: Face
Skin Concerns: Dullness, Firmness & Elasticity
Product Type: Solid
Formulation: Liquid

'''
# Clean skin_type column
# Delete rows where cell under skin_type column starts with certain words (wrong info)

search ="Function:"
df = df[~df["skin_type"].str.startswith(search)]

search ="Skin Concerns:"
df = df[~df["skin_type"].str.startswith(search)]

search ="Product Type:"
df = df[~df["skin_type"].str.startswith(search)]

search ="Formulation:"
df = df[~df["skin_type"].str.startswith(search)]

In [8]:
# Clean label column

# df.Label[df['Label'] == 'moisturizing-cream-oils-mists'] = str('Moisturizer')
# df.Label[df['Label'] == 'cleanser'] = str('Cleanser')
# df.Label[df['Label'] == 'facial-treatments'] = str('Treatment')
# df.Label[df['Label'] == 'face-mask'] = str('Face Mask')
# df.Label[df['Label'] == 'eye-treatment-dark-circle-treatment'] = str('Eye cream')
# df.Label[df['Label'] == 'sunscreen-sun-protection'] = str('Sun protect')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 417 entries, 0 to 534
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Label        417 non-null    object 
 1   URL          417 non-null    object 
 2   brand        417 non-null    object 
 3   name         417 non-null    object 
 4   price        417 non-null    object 
 5   rank         417 non-null    float64
 6   skin_type    417 non-null    object 
 7   ingredients  417 non-null    object 
dtypes: float64(1), object(7)
memory usage: 29.3+ KB


In [10]:
# Clean Name column
# Drop duplicated item:

df_2 = df['name'].drop_duplicates()
df = df.loc[df_2.index, :].reset_index()



In [11]:
# Clean URL column 
df.drop(['URL', 'index'], axis = 1, inplace = True)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403 entries, 0 to 402
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Label        403 non-null    object 
 1   brand        403 non-null    object 
 2   name         403 non-null    object 
 3   price        403 non-null    object 
 4   rank         403 non-null    float64
 5   skin_type    403 non-null    object 
 6   ingredients  403 non-null    object 
dtypes: float64(1), object(6)
memory usage: 22.2+ KB


In [13]:
# Clean rank column

df['rank'].fillna(0, inplace = True)

In [14]:
# Clean price column

pattern = re.compile(r"(\d+).\d+")
for i in range(len(df)):
    df['price'][i] = re.findall(pattern, df['price'][i])[0]

df['price'] = pd.to_numeric(df['price'])

C:\Users\storm\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [15]:
df.head()

,Label,brand,name,price,rank,skin_type,ingredients
0,cleanser-and-exfoliator,FRESH,Soy Face Cleanser,113,4.5,"Skin Type: Combination, Dry, Normal, Oily, Sen...","Aqua (Water), Coco-Glucoside, Glycerin, Butyle..."
1,cleanser-and-exfoliator,INDIE LEE,Brightening Cleanser,50,4.5,"Skin Type: Combination, Dry, Normal, Oily, Sen...","Water (Aqua/Eau), Decyl Glucoside (Veg. Oils &..."
2,cleanser-and-exfoliator,TARTE,Sea Deep Dive Makeup Removing Gel Cleanser,40,4.5,"Skin Type: Combination, Dry, Normal, Oily, Sen...","Water/Aqua/Eau, cocamidopropyl betaine, disodi..."
3,cleanser-and-exfoliator,FRESH,Rose Cleansing Foam • 120ml,47,4.0,"Skin Type: Combination, Dry, Normal, Oily, Sen...","Aqua (Water), Butylene Glycol, Decyl Glucoside..."
4,cleanser-and-exfoliator,DR. ROEBUCKS,Noosa Nourishing Crème Cleanser • 100ml,30,4.5,"Skin Type: Dry, Normal, Oily, Sensitive","Aqua/Water/Eau, Stearic Acid, Cetearyl Alcohol..."


In [16]:
# Clean skin_type column

pattern = 'Skin Type: '
for i in range(len(df)):
    df['skin_type'][i] = re.sub(pattern, '', df['skin_type'][i])

C:\Users\storm\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [17]:
df.head()

,Label,brand,name,price,rank,skin_type,ingredients
0,cleanser-and-exfoliator,FRESH,Soy Face Cleanser,113,4.5,"Combination, Dry, Normal, Oily, Sensitive","Aqua (Water), Coco-Glucoside, Glycerin, Butyle..."
1,cleanser-and-exfoliator,INDIE LEE,Brightening Cleanser,50,4.5,"Combination, Dry, Normal, Oily, Sensitive","Water (Aqua/Eau), Decyl Glucoside (Veg. Oils &..."
2,cleanser-and-exfoliator,TARTE,Sea Deep Dive Makeup Removing Gel Cleanser,40,4.5,"Combination, Dry, Normal, Oily, Sensitive","Water/Aqua/Eau, cocamidopropyl betaine, disodi..."
3,cleanser-and-exfoliator,FRESH,Rose Cleansing Foam • 120ml,47,4.0,"Combination, Dry, Normal, Oily, Sensitive","Aqua (Water), Butylene Glycol, Decyl Glucoside..."
4,cleanser-and-exfoliator,DR. ROEBUCKS,Noosa Nourishing Crème Cleanser • 100ml,30,4.5,"Dry, Normal, Oily, Sensitive","Aqua/Water/Eau, Stearic Acid, Cetearyl Alcohol..."


In [18]:
# list column dummies

df_3 = df['skin_type'].str.get_dummies(sep=', ')
df_4 = df.join(df_3).drop('skin_type', axis = 1)

In [19]:
df_3.head()

,Combination,Dry,Normal,Oily,Sensitive
0,1,1,1,1,1
1,1,1,1,1,1
2,1,1,1,1,1
3,1,1,1,1,1
4,0,1,1,1,1


In [20]:
#df.to_csv('test_check.csv', encoding = 'utf-8-sig', index = False)

In [21]:
#df_3.to_csv('seesee4.csv', encoding = 'utf-8-sig', index = False)

In [22]:
#df_4.to_csv('sephora_SG_FIREFOX_dataset_clean_SENSITIVE SKIN.csv', encoding = 'utf-8-sig', index = False)

In [23]:
df_4.to_csv('sephora_SG_FIREFOX_dataset_clean.csv', encoding = 'utf-8-sig', index = False)